# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""../../../../da…","""91537983947420…",1.2927e19,"""17787783548334…","""13949120975360…","""16483003167655…",false
"""../../../../da…","""12192997146143…",1.4856e19,"""15717325996882…","""13949120975360…","""14942267818369…",true
"""../../../../da…","""85551156509646…",5.6458e17,"""15038183838131…","""62803811632625…","""16483003167655…",false
"""../../../../da…","""16597045212157…",4.0723e18,"""15717325996882…","""49175861167714…","""16483003167655…",false
"""../../../../da…","""14960875811967…",1.4677e19,"""17787783548334…","""49175861167714…","""68286768516717…",true
"""../../../../da…","""41065761878276…",1.6330e19,"""15038183838131…","""34706048914450…",null,false
"""../../../../da…","""45131339778982…",8.7675e18,"""15038183838131…","""49175861167714…","""68286768516717…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_City': 'PREDICTOR_1',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../da…","""48930292713324…",1.2927e19,"""49721894291757…","""FirstMortgage3…","""76798460221290…","""Rejected"""
"""../../../../da…","""17802045567706…",1.4856e19,"""17558239667950…","""FirstMortgage3…","""17052231426227…","""Accepted"""
"""../../../../da…","""23899849360544…",5.6458e17,"""87334557199652…","""MoneyMarketSav…","""76798460221290…","""Rejected"""
"""../../../../da…","""13194037287397…",4.0723e18,"""17558239667950…","""BasicChecking""","""76798460221290…","""Rejected"""
"""../../../../da…","""24943622807458…",1.4677e19,"""49721894291757…","""BasicChecking""","""14850802166340…","""Accepted"""
"""../../../../da…","""14248744326889…",1.6330e19,"""87334557199652…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""12956916011414…",8.7675e18,"""87334557199652…","""BasicChecking""","""14850802166340…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../da…","""28675432219020…",1.2927e19,"""10663399084957…","""FirstMortgage3…","""17803072799183…","""Rejected"""
"""../../../../da…","""25550751914374…",1.4856e19,"""76916091776148…","""FirstMortgage3…","""15418160271295…","""Accepted"""
"""../../../../da…","""60884468447836…",5.6458e17,"""41231065076618…","""MoneyMarketSav…","""17803072799183…","""Rejected"""
"""../../../../da…","""11873817884511…",4.0723e18,"""76916091776148…","""BasicChecking""","""17803072799183…","""Rejected"""
"""../../../../da…","""56481694702543…",1.4677e19,"""10663399084957…","""BasicChecking""","""11124523082494…","""Accepted"""
"""../../../../da…","""86398635311454…",1.6330e19,"""41231065076618…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""98978223885433…",8.7675e18,"""41231065076618…","""BasicChecking""","""11124523082494…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_City=PREDICTOR_1
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""78311278142465…",1.2927e19,"""62157754303957…","""FirstMortgage3…","""17740463836246…","""Rejected"""
"""76814019049289…",1.4856e19,"""12165907458774…","""FirstMortgage3…","""10658292327239…","""Accepted"""
"""37605599286607…",5.6458e17,"""11996928021266…","""MoneyMarketSav…","""17740463836246…","""Rejected"""
"""74546663459619…",4.0723e18,"""12165907458774…","""BasicChecking""","""17740463836246…","""Rejected"""
"""93837840230516…",1.4677e19,"""62157754303957…","""BasicChecking""","""69561629516627…","""Accepted"""
"""14559252798949…",1.6330e19,"""11996928021266…","""UPlusFinPerson…",null,"""Rejected"""
"""71963126700945…",8.7675e18,"""11996928021266…","""BasicChecking""","""69561629516627…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'